### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os
from faker import Faker
from datetime import date, timedelta
from random import choices

### Simulate Data: 100,000 entries

In [2]:
# size of simulation
k = int(1e5)

In [3]:
# date ranges
beginning, end = np.datetime64('2017-01-01'), np.datetime64('2022-12-31')

# added_date
added_date = [beginning]

# loop to get each date
while beginning != end:
    beginning += np.timedelta64(1, 'D')
    added_date.append(beginning)

# random K dates from pack
added_date = choices(added_date, k=k)

In [4]:
# task duration
real_duration = np.random.randint(0, 20, k)

In [5]:
# end date
end_date = []

for (i, j) in zip(added_date, real_duration):
        one_date = np.datetime64(i) + np.timedelta64(j, 'D')
        end_date.append(one_date)

In [6]:
# test to see if it worked
added_date[50], real_duration[50], end_date[50]

(numpy.datetime64('2019-10-17'), 17, numpy.datetime64('2019-11-03'))

In [7]:
# test size of end_date list
len(end_date)

100000

In [8]:
# test size of date_added list
len(added_date)

100000

> Note: Getting [total business days](https://www.geeksforgeeks.org/python-program-to-get-total-business-days-between-two-dates/) using [NumPy](https://numpy.org/doc/stable/reference/generated/numpy.busday_count.html)

In [9]:
# calculate real duration once again, but this
# time around, counting business days only
real_duration = []

for (i, j) in zip(added_date, end_date):
    one_duration = np.busday_count(i, j + 1)
    real_duration.append(one_duration)

In [10]:
# test to see if it worked
added_date[50], real_duration[50], end_date[50]

(numpy.datetime64('2019-10-17'), 12, numpy.datetime64('2019-11-03'))

In [11]:
# estimated_duration
est_duration = np.random.randint(0, 20, k)

In [12]:
# list of hypothetical softwares
software_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [13]:
# get 1e4 entries
software_k = np.random.choice(software_list, size=k)

> Note: [Faker](https://faker.readthedocs.io/en/master/index.html)'s documentation

In [14]:
# for the dev names, I'll use the Faker library
fake = Faker(['pt_BR'])

# empty list
devs = []

for _ in range(10):
    dev = fake.first_name_female()
    devs.append(dev)

# get k entries
dev_k = np.random.choice(devs, size=k)

In [15]:
# status
status = ['Não Iniciado', 'Em Desenvolvimento', 'Em Homologação', 'Pausado', 'Concluído']

# get k entries
status_k = np.random.choice(status, k, p=[0.2, 0.1, 0.1, 0.1, 0.5])

# test status_k size
len(status_k)

100000

In [16]:
# returns (number of times an item returned to developer)
possible_returns = [0, 1, 2, 3]

# k-sized list of possible returns
# notice 0 has the highest probability
returns_k = np.random.choice(possible_returns, k, p=[0.65, 0.2, 0.1, 0.05])

In [17]:
# squad responsible for the ticket
squad = ['Epsilon', 'Theta', 'Zeta', 'Kappa', 'Chi']

# get k entries
squad_k = np.random.choice(squad, k, p=[0.3, 0.2, 0.05, 0.4, 0.05])

In [18]:
# get client list using Faker
# empty list
clients = []

for _ in range(200):
    client = fake.company()
    clients.append(client)

# get k entries
clients_k = np.random.choice(clients, size=k)

In [19]:
df = pd.DataFrame({'estimated_duration': est_duration,
                  'added_date': added_date,
                  'end_date': end_date,
                  'real_duration': real_duration,
                  'returns': returns_k,
                  'software': software_k,
                  'dev': dev_k,
                  'squad': squad_k,
                  'client': clients_k,
                  'status': status_k})

In [20]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   estimated_duration  100000 non-null  int32         
 1   added_date          100000 non-null  datetime64[ns]
 2   end_date            100000 non-null  datetime64[ns]
 3   real_duration       100000 non-null  int64         
 4   returns             100000 non-null  int32         
 5   software            100000 non-null  object        
 6   dev                 100000 non-null  object        
 7   squad               100000 non-null  object        
 8   client              100000 non-null  object        
 9   status              100000 non-null  object        
dtypes: datetime64[ns](2), int32(2), int64(1), object(5)
memory usage: 6.9+ MB
None


,estimated_duration,added_date,end_date,real_duration,returns,software,dev,squad,client,status
0,19,2018-02-09,2018-02-12,2,1,B,Luna,Epsilon,Barbosa,Não Iniciado
1,2,2018-02-18,2018-03-07,13,0,F,Marcela,Kappa,Correia,Em Desenvolvimento
2,6,2022-09-25,2022-09-25,0,0,F,Clarice,Kappa,Nunes,Concluído
3,10,2019-08-08,2019-08-25,12,0,E,Marcela,Kappa,Rocha S/A,Concluído
4,5,2021-10-12,2021-10-13,2,0,E,Luna,Kappa,da Rosa - ME,Em Homologação


### Data Wrangling

In [21]:
# if status is different from 'concluído' or 'em homologação', 
# then logically the dev isn't done yet, thus there should be
# no end date nor real duration for the task
df.loc[(df['status'] != "Concluído") & (df['status'] != "Em Homologação"), 'end_date'] = np.nan
df.loc[(df['status'] != "Concluído") & (df['status'] != "Em Homologação"), 'real_duration'] = np.nan

In [22]:
df.loc[(df['status'] == "Em Desenvolvimento"), "start_date"] = df['added_date']

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   estimated_duration  100000 non-null  int32         
 1   added_date          100000 non-null  datetime64[ns]
 2   end_date            59990 non-null   datetime64[ns]
 3   real_duration       59990 non-null   float64       
 4   returns             100000 non-null  int32         
 5   software            100000 non-null  object        
 6   dev                 100000 non-null  object        
 7   squad               100000 non-null  object        
 8   client              100000 non-null  object        
 9   status              100000 non-null  object        
 10  start_date          9929 non-null    datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int32(2), object(5)
memory usage: 7.6+ MB


In [24]:
df.query('status == "Em Desenvolvimento"')

,estimated_duration,added_date,end_date,real_duration,returns,software,dev,squad,client,status,start_date
1,2,2018-02-18,NaT,NaN,0,F,Marcela,Kappa,Correia,Em Desenvolvimento,2018-02-18
29,19,2021-06-06,NaT,NaN,1,D,Evelyn,Theta,Viana Cavalcanti Ltda.,Em Desenvolvimento,2021-06-06
36,5,2022-03-28,NaT,NaN,0,D,Clarice,Kappa,Rocha Ltda.,Em Desenvolvimento,2022-03-28
53,6,2017-06-05,NaT,NaN,2,F,Júlia,Epsilon,Lopes,Em Desenvolvimento,2017-06-05
74,16,2022-01-01,NaT,NaN,0,C,Evelyn,Kappa,Costela,Em Desenvolvimento,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...
99960,18,2019-03-17,NaT,NaN,0,E,Evelyn,Kappa,Nascimento S.A.,Em Desenvolvimento,2019-03-17
99968,8,2017-03-23,NaT,NaN,1,E,Amanda,Kappa,Ramos S/A,Em Desenvolvimento,2017-03-23
99994,10,2022-07-16,NaT,NaN,0,A,Júlia,Chi,Aragão Costela Ltda.,Em Desenvolvimento,2022-07-16
99995,15,2021-06-21,NaT,NaN,1,E,Luna,Epsilon,Correia,Em Desenvolvimento,2021-06-21


In [25]:
# create directory
folder_name = 'data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [26]:
df.to_csv('data/simulated-data.csv', index=False, encoding='latin1')